$$
\def\CC{\bf C}
\def\QQ{\bf Q}
\def\RR{\bf R}
\def\ZZ{\bf Z}
\def\NN{\bf N}
$$
# Tutorial: Real and complex numbers in Sage

Authors  
-   Thierry Monteil

License  
CC BY-SA 3.0

Computers can not represent all real or complex numbers. With unlimited
memory, you could represent all integers and rationals but not the real
numbers since there are uncountably many of them. Sage proposes
different solutions to make computations with real numbers. Each
different way corresponds to a parent in the categories `Fields` or
`Rings` though they do not necessarily satisfy their respective
axiomatic. These different solutions have very different features and it
is important to know to which kind of problem which representation to
choose.

You should know a bit about coercion before starting this tutorial, so
it is advised to read the `parent_element_coercion.en.rst` worksheet
first.

## The big zoo

Here is a rough qualitative summary of the main representations that
will appear in this tutorial:

| Kind      | Name                                                                                  | Short                  | Complex equivalent                                                                                | Short                  | Reliable                        | Fast                      | Expressive                | Exact           | Precision                                                                                                    |     |
|-----------|---------------------------------------------------------------------------------------|------------------------|---------------------------------------------------------------------------------------------------|------------------------|---------------------------------|---------------------------|---------------------------|-----------------|--------------------------------------------------------------------------------------------------------------|-----|
| Numerical | `RealDoubleField()` `RealField(prec)` `RealIntervalField(prec)` `RealBallField(prec)` | `RDF` `RR` `RIF` `RBF` | `ComplexDoubleField()` `ComplexField(prec)` `ComplexIntervalField(prec)` `ComplexBallField(prec)` | `CDF` `CC` `CIF` `CBF` | `+++` `+++` `+++++` `+++++`     | `+++++` `+++` `+++` `+++` | `+` `++` `++` `++`        | No No No No     | bounded by 53 bits finite unbounded non-adaptive finite unbounded non-adaptive finite unbounded non-adaptive |     |
| Algebraic | `AlgebraicRealField()` `NumberField(poly)` `QuadraticField(n)` `RationalField()`      | `AA` none none `QQ`    | `AlgebraicField()` itself itself `QuadraticField(-1)`                                             | `QQbar` none none none | `+++++` `+++++` `+++++` `+++++` | `+` `++` `+++` `++++`     | `++++` `++++` `+++` `+++` | Yes Yes Yes Yes | infinite infinite infinite infinite                                                                          |     |
| Symbolic  | `SymbolicRing()`                                                                      | `SR`                   | itself                                                                                            | `SR`                   | `+`                             | `+`                       | `+++++`                   | No              | depends on construction                                                                                      |     |

Numerical representations are fast but inexact, they require some
special care since the precision may become fuzzy along a computation.

Algebraic representations are exact and reliable, but slower and able to
represent only a class of numbers.

Symbolic representations are very versatile, e.g. they can represent
numbers such as $\sqrt{log(\pi)+sin(42)}$, but slow and sometimes not
reliable.

Let us inspect each kind of representation further.

**TODO**

Find a better word for "kind" that is not "type", "group", "category",
"class" (perhaps "taxon", "family", "sort" ?).

## Numerical representations

### Floating-point numbers

Floating-point numbers are real numbers of the form
$(-1)^s 0.d_0 d_1 d_2 \dots
d_{p-1} 2^{e}$ where:

-   $s \in \{0,1\}$ is the *sign*,
-   $d_i \in \{0,1\}$ (with $d_0=1$) are the *digits*,
-   the integer $d_0 d_1 d_2 \dots d_{p-1}$ is the *mantissa*,
-   $p$ is the *precision*,
-   $e$ is the *exponent*.

Floating-point numbers of given precision do not form a field in the
mathematical sense since they are not stable under the classical
operations. So when adding or multiplying two floating-point numbers,
Sage has to round the result to return a floating-point number. In
particular, the results are approximate and the error can grow along a
computation.

Some first warnings about floating-point arithmetics can be found on the
guide: <http://floating-point-gui.de/>

Error propagation is sometimes unavoidable, but it is possible to know a
bound on it by using the following secure fields:

There exists various floating-point implementations of numbers in Sage:

#### Real Double Field

Elements of `RDF = RealDoubleField()` are the floating-point numbers in
double precision from the processor (those you use when you code in C,
similar to Python $float$), see <span
role="mod">sage.rings.real\_double</span>. The computations using those
numbers are very fast and most optimized libraries that work with
floating-point numbers use this representation, so you will benefit from
them if the number involved in your floating-point constructions
(matrices, polynomials,...) belong to `RDF`. The mantissa has 53 bits of
precision, the exponent is encoded on 11 bits from -1023 to 1024. Note
that since the first bit of the mantissa is always $1$, we only have to
store $1 + 52 + 11 = 64$ bits as expected. During the computations, the
rounding is done toward the closest floating-point number.

#### Real Fields

Elements of `RealField(prec)` are the floating-point numbers with `prec`
bits of precision, "emulated" by the MPFR library. By default, the
mantissa has $53$ bits of precision (and `RealField(prec=53)` can be
shortened as `RR`), the exponent is encoded on 32 bits (so that you will
unlikely overflow), see <span role="mod">sage.rings.real\_mpfr</span>.
The computations are slower than in `RDF` and most algorithms on objects
whose base ring is `RealField(prec)` will be pretty naive. However, it
is possible to extend the precision as well as the type of rounding
(towards $+\infty$, towards $-\infty$, towards $0$), for example
`RealField(100, rnd='RNDZ')` is the "field" of real floating-point
numbers with 100 bits of precision with rounding towards zero.

Note

While sometimes needed, increasing the precision also increases the
computation time.

Warning

Note that despite its generic name, `RealField(prec)` or `RR` is *not*
an abstraction of the field of real numbers, as can be seen by the
following section.

#### Unreal floats

Besides not representing all real numbers, `RDF` and `RR` contain three
additional unreal elements that allow the `(+,-,*,/)` operations to be
always defined (`NaN` stands for *Not a Number*):

In [ ]:
RDF(0) / RDF(0)

NaN


In [ ]:
RDF(1) / RDF(0)

+infinity


In [ ]:
RR(-1) / RR(0)

-infinity


In [ ]:
RDF(infinity) - RDF(infinity)

NaN


In [ ]:
NaN in RR and Infinity in RR and -Infinity in RR

True

#### Back and forth in precision

Note that it is is general useless to improve the precision during a
computation. This explains why the coercion is done toward the smallest
precision:

In [ ]:
R = RealField(200)
S = RealField(100)
a = R(2).sqrt()
b = S(pi)
a+b == R(sqrt(2)+pi)

True

In [ ]:
a+R(b) == R(sqrt(2)+pi)

False

Note

For each element of those two fields, there exists a
`.sign_mantissa_exponent()` method that returns the corresponding
triple.

### Floating-point numbers with a control on the imprecision

#### Real Interval Fields

Elements of `RealIntervalField(prec)` are pairs of elements of
`RealField(prec)`, which should be interpreted as the two endpoints of
an interval containing the represented real number.
`RealIntervalField(53)` can be shortened as `RIF`.

Operations defined over the `RealIntervalField` implementation must
offer the *containment guaranty*: for every mathematical function
$f : \mathbb{R}^d \to
\mathbb{R}$ that admits an implementation `f_rif` over some
`RealIntervalField(prec)`, we have the property that for every element
`x = [(a_0, b_0), (a_1,b_1), ..., (a_(d-1),b_(d-1))]` in
`RealIntervalField(prec)^d`, if `f_rif(x) = [a', b']`, then
$f([a_0, b_0]
\times [a_1,b_1] \times \dots \times [a_(d-1),b_(d-1)])\subseteq [a', b']$.
In particular, the left endpoint of the interval is rounded towards
$-\infty$ and the right endpoint is rounded towards $+\infty$

#### Real Ball Fields

Elements of `RealBallField(prec)` are pairs `(c,r)`, where `c` is a
floating-point number with `prec` bits of precision, and where `r` is a
floating-point number with 30 bits of precision. The high precision `c`
and the low precision number `r` should be understood as the center and
the radius (respectively) of a ball containing the represented real
number. `RealBallField(53)` can be shortened as `RBF`.

Operations defined over the `RealBallField` implementation must offer a
similar containment guaranty as for `RealIntervalField` (replace
intervals with balls).

#### Comparison between both representations

The main advantage of the ball implementation over interval arithmetics
is that, when the precision is high, the error is still encoded with a
low precision (since only the order of magnitude matters for the error
bounds), so that the loss of speed compared to non-guaranteed
floating-point arithmetics with the same precision is about
$1+\varepsilon$ (and not $2$ as in the case of interval arithmetics,
where two precise numbers have to be computed).

Also, it is closer to the mathematical $\varepsilon-\delta$ definitions
and distance-based error estimations.

However, for large intervals and for evaluating monotonic functions,
interval arithmetics is usually more precise and easier to implement
(just round the images of the endpoints towards the right direction). It
is also well-adapted for implementing dichotomy-based precision
refinements.

### Which numerical representation to choose

Sage interfaces various libraries written in C, C++, Fortran, etc. which
use the processor double precision floats. Hence, the use of `RDF` has
the advantage of being faster and allowing the use of those libraries.

If you use other floating-point representations, Sage will mostly use
generic algorithms, which are usually slower and numerically less
stable. However, those representations allow you to express real numbers
with a higher precision.

Regarding speed, `RealBallField(prec)` and `RealField(prec)` are
similar, while `RealIntervalField(prec)` is twice slower. However, the
`RealBallField(prec)` is a new feature of Sage, hence some methods might
raise a `NotImplementedError`.

**Summary**

So, if you need speed and the power of Sage libraries: use `RDF`.

If you need more precision, start with `RealBallField(prec)`, and try
`RealIntervalField(prec)` in case the imprecision becomes too large
(alternatively: increase the precision or try to think about the order
and the expanding nature of your operations). Use `RealField(prec)` if
the methods you wanted to use with `RealBallField(prec)` are not
implemented.

### Exercises

#### Binomial

From the three following values:

In [ ]:
a = 1.0
b = 10.0^4
c = 1.0

solve the $ax^2+bx+c = 0$ equation by the method you learned in
highschool involving discriminant, and compare the sum and product of
the solution you found to their theoretical values $-b/a$ and $c/a$.

In [ ]:
# edit here

Same question by letting Sage solve this equation for you.

In [ ]:
# edit here

Even on such a simple computation, you have been victim of a phenomenon
of *error amplification*. Another striking example is the following:

In [ ]:
a = 10000.0
b = -9999.5
c = 0.1
a+c+b

0.600000000000364

In [ ]:
a+b+c

0.600000000000000

Note that, by construction and following the IEEE 754 specification, the
`+` (and `*`) law is commutative in `RDF` and `RR` :

In [ ]:
all([a+b==b+a, a+c==c+a, b+c==c+b, (a+b)+c==c+(a+b), (a+c)+b==b+(a+c), (b+c)+a==a+(b+c)])

True

Actually the associativity is not satisfied here:

In [ ]:
(a+b)+c == a+(b+c)

False

Try to find, by random sampling, 3 elements $a,b,c$ in `RDF` such that
`(a+b)+c != a+(b+c)` :

In [ ]:
# edit here

Why is it so hard ? What makes the previous example work ?

#### Interval arithmetics

"Prove" using `RealIntervalField` that $cos(2\pi)=1$ with a high
precision (say $10^{-100}$).

In [ ]:
# edit here 

Explain the following behaviour:

In [ ]:
sqrt(0.1)^2 == 0.1

True

In [ ]:
sqrt(RIF(0.1))^2 == RIF(0.1)

False

Provide a relation between `sqrt(RIF(0.1))^2` and `RIF(0.1)`

In [ ]:
# edit here

Note that the ordering of the computations has an influence on the
result, but also on its precision. With the previous example, we have:

In [ ]:
a = RIF(10000.0)
b = RIF(9999.5)
c = RIF(0.1)
a+c-b

0.60000000000?

In [ ]:
a-b+c

0.6000000000000000?


In [ ]:
(a+c-b).endpoints()

(0.599999999998544, 0.600000000000364)

In [ ]:
(a+c-b).absolute_diameter()

1.81898940354586e-12


In [ ]:
(a-b+c).endpoints()

(0.599999999999999, 0.600000000000001)

In [ ]:
(a-b+c).absolute_diameter()

1.11022302462516e-16

Before evaluating them, could you predict and explain the behaviour of
the following commands ?

In [ ]:
# 1/2 in RIF

In [ ]:
# 1/3 in RIF

*Hint*: you can look at the documentation of the `.__contains__()`
method of `RIF`.

A Sage developer wants to write the real function $f$ that maps $0.01$
to $1$ and that is the identity elsewhere, and suggests the following
implementation:

In [ ]:
def f_unstable(x):
    if x == 0.01:
        return 1
    else:
        return x

Prove that this implementation does not offers the containment guaranty
when used on the `RealIntervalField`.

Could you help this lost developer to find a correct implementation so
that it could be included in Sage source code ?

In [ ]:
def f_correct(x):
    # edit here

#### Divergent series ?

The series
$u_n := 1 + 1/2 + 1/3 + 1/4 + \dots + 1/n = \sum_{k=1}^{n} 1/k$ is known
to diverge on $\mathbb{R}$.

Explore its convergence on some $Real Fields$ (start with small
precision). Explore the difference with the (mathematically equal)
series $v_n := 1/n +
1/(n-1) + ... + 1/2 + 1$.

Explain, formulate conjectures and try to prove them.

In [ ]:
# edit here

#### A picture

Draw (by hand) a picture of the elements of `RealField(3)`.

#### Convergence of a Markov Chain

Let $M$ be the rational bistochastic matrix defined by:

In [ ]:
M = matrix(QQ, [[1/6, 1/2, 1/3], [1/2, 1/4, 1/4], [1/3, 1/4, 5/12]])
M

[ 1/6  1/2  1/3]
[ 1/2  1/4  1/4]
[ 1/3  1/4 5/12]

Could you guess the limit of $M^n$ when $n$ goes to $+\infty$ ?

In [ ]:
# edit here

Could you prove that this limit exists and give its exact value ?

In [ ]:
# edit here

#### Repulsive and attractive fixed points

Define the maps $f:x\to 4x-1$ and $g:x\to x/3+1$.

In [ ]:
# edit here

Let $u_n$ and $v_n$ be the sequences defined recursively by
$u_0 = RDF(1/3)$, $u_{n+1} = f(u_n)$ and $v_0 = RDF(3/2)$,
$v_{n+1} = g(v_n)$.

Compute the first 100 values of the sequences $u_n$ and $v_n$.

In [ ]:
# edit here

For which reason you should not worry to find the fixed point of the
function $h:x\to sin(x + 1/2)$ ? Compute the value of this fixed point
with 50 bits of precision.

In [ ]:
# edit here

#### Decimals of pi

What is the 150th decimal of $\pi$ ?

In [ ]:
# edit here

#### Money, money, money

In [ ]:
one_cent = 0.01
my_money = 0
for i in range(100):
    my_money += one_cent
my_money > 1

What is the origin of this behaviour ?

How to fix the issue in Sage (respecively un pure Python, compare both)
?

In [ ]:
# edit here

Which risk does computer science take if finance and high-frequency
trading stays on power for a while ?

Provide the first 30 bits of the binary expansion of $0.01$.

In [ ]:
# edit here

#### Huge matrix computation

Construct a dense matrix of size `500*500` with real coefficients
randomly distributed in $[0,1]$.

In [ ]:
# edit here

Take the 100th power of this matrix in less than one second.

In [ ]:
# edit here

Comment.

#### Exponents

In [ ]:
RDF(1/2^10000) == 0

True

In [ ]:
RR(1/2^10000) == 0

False

Comment.

#### A strange impression

In [ ]:
RR(1/pi)

0.318309886183791

In [ ]:
RDF(1/pi)

0.3183098861837907

Before evaluating the next command, could you tell what will be the
answer of the following command ?

In [ ]:
# RR(1/pi) == RDF(1/pi)

What is the cause of the behaviour of the first two commands ?

#### Rounding

Let us define:

In [ ]:
n = 5.0000000000000001

Explain each of the following behaviours:

In [ ]:
n

5.000000000000000

In [ ]:
n.str(truncate=False)

'5.000000000000000111'

In [ ]:
5.000000000000000111 == 5.0000000000000001

## Algebraic (exact) representations

Some subfields of the real numbers can be represented in an exact way in
Sage. They are pretty safe (no rounding problem, well defined semantics)
through sometimes slow.

As a general rule, the smallest the field is (for the inclusion), the
fastest it is, which means that, in principle, working within rational
field is faster than within quadratic fields, which is faster than
within cyclotomic fields, which is faster than within number fields,
which is faster that within the universal cyclotomic field, that is
faster than within the whole algebraic field.

Here is a short overview:

### Rational numbers

`QQ = RationalField()` stands for the field of rational numbers.
Computations are exact and only limited by the available memory. You
should note that the numerators/denominators could become very large
along a computation, which might slow down this latter.

In [ ]:
(123/321)^10000

It is possible to "approximate" a floating-point real number by rational
numbers with the methods `nearby_rational()`, `simplest_rational()`,
`exact_rational()`.

In [ ]:
# edit here

If `x` belongs to `RR`, what is the result of `QQ(x)` ? Is it a coercion
?

See the page
<http://doc.sagemath.org/html/en/reference/rings_standard/sage/rings/rational_field.html>

### Algebraic number field

`AA = AlgebraicRealField()` stands for te field of real algebraic
numbers.

In [ ]:
a = AA(2).sqrt() + AA(7).sqrt()
a

4.059964873437685?

Here, the number `a` is *printed* as its approximated numerical value,
but the question mark at the end indicates that it is not internally a
floating-point number.

Basically, an algebraic number is represented as the root of an integer
polynomial, and a small interval surrounding the appropriate root:

In [ ]:
a.minpoly()

x^4 - 18*x^2 + 25


In [ ]:
a.n(digits=50)

4.0599648734376856393033044778489585042799310584594

In [ ]:
a.interval_diameter(1e-40)

4.059964873437685639303304477848958504279931058459398253545014197191801301693?

Computing such a representation is costly, hence Sage is lazy and
computes it only when needed:

In [ ]:
b = QQbar(2).sqrt()
c = b^2
c

2.000000000000000?

Here, Sage did not compute the minimal polynomial nor an enclosing
interval of `c`, it just knows `c` as "the square of `b`", in
particular, it does not know yet that `c` is the integer `2`. However,
such an "exactification" is computed when required (and kept
internally):

In [ ]:
c.is_integer()

True

In [ ]:
c

2

You can force the exactification with the `exactify` method.

For more details, see the page
<http://doc.sagemath.org/html/en/reference/number_fields/sage/rings/qqbar.html>

### Number fields

See the pages:

-   <http://doc.sagemath.org/html/en/reference/number_fields/index.html>
-   <http://doc.sagemath.org/html/en/constructions/number_fields.html>
-   <http://doc.sagemath.org/html/en/reference/number_fields/sage/rings/number_field/number_field.html>

Warning

You should be aware that number field are purely algebraic objects that
are *not* embedded into the complex plane by default. Do to this, you
have to define the embedding you want to work with (if you need one).

Note that the quadratic number fields have a dedicated faster
implementation.

What is the difference between those two objects ?

In [ ]:
q = QuadraticField(-1,'I')
qq = QQ[I]

### Cyclotomic fields (and the universal cyclotomic field)

See the pages:

-   <http://doc.sagemath.org/html/en/reference/number_fields/sage/rings/number_field/number_field.html#sage.rings.number_field.number_field.CyclotomicFieldFactory>
-   <http://doc.sagemath.org/html/en/reference/number_fields/sage/rings/universal_cyclotomic_field.html>

## Symbolic representations

`SR = sage.symbolic.ring.SymbolicRing()` is the set of symbolic
expressions.

`SR` is pretty fuzzy but quite handy (here expressivity is prefered over
standardization and the number of real numbers representable within `SR`
is constantly increasing). This "ring" contains most mathematical
constants such as $e$, $\pi$, $cos(12)$, $\sqrt{2}$, $\gamma$ (Euler
constant), ... and allows some computations such as:

In [ ]:
e^(I*pi)

-1

You should not consider `SR` as reliable:

In [ ]:
bool(567663097408 * pi - 1783366216531 == 0)

True

Warning

`SR` also contains symbolic expressions that are not real or complex
numbers:

In [ ]:
e = cos(x)^2
e.derivative()

-2*cos(x)*sin(x)

In [ ]:
_ in SR

True

`SR` represents numbers as symbolic expressions, whose length (as a
formula) can grow along a computation. This phenomenon can slow down the
computations. If our aim is to end with a numerical value, you should
use a floating-point representation, as they keep a constant length.

In [ ]:
a = exp(2)
f = lambda x : sqrt(x+1)^x
for i in range(10):   
    a = f(a)
a

Of course, using floating-point arithmetics will not necessarily solve
all your problems since you will have to ensure the numerical stability
of such iterated computation (see the sections above).

`SR` always answer, even when it does not know the answer, for example:

In [ ]:
# missing example

Hence, computations involving the symbolic ring should not be considered
as reliable. In particular, `SR` is a dead-end for the coercion among
numbers (and can add pears and apples formally):

In [ ]:
0.1 + pi + QQbar(2).sqrt()

pi + 1.51421356237310

In [ ]:
_.parent()

Symbolic Ring

Here are some issues that can appear with the symbolic ring, when you
can represent your numbers in a different way, we strongly encourage you
to do so:

-   <https://ask.sagemath.org/questions/tags:symbolic_issue/>
-   <https://trac.sagemath.org/wiki/symbolics>
-   <https://ask.sagemath.org/question/2469/quadratic-number-fields>
-   <https://ask.sagemath.org/question/2561/is-it-a-bug>
-   <https://ask.sagemath.org/question/2518/get-variants-of-complex-cube-root>
-   <https://ask.sagemath.org/question/2734/computing-the-digist-of-pi>

## Conversions, bridges, exactification

**TODO**

Should this be a separate section, or should it appear in the
rational/algebraic sections ?

Along a computation, coercion goes towards the less precise numbers:

In [ ]:
a = 1/10 ; a

1/10

In [ ]:
a.parent()

Rational Field

In [ ]:
b = QQbar(2).sqrt() ; b

1.414213562373095?

In [ ]:
b.parent()

Algebraic Field

In [ ]:
c = a+b ; c

1.514213562373096?

In [ ]:
c.parent()

Algebraic Field

In [ ]:
d = RDF(0.1) ; d

0.1

In [ ]:
d.parent()

Real Double Field

In [ ]:
e = c+d ; e

1.614213562373095

In [ ]:
e.parent()

Complex Double Field

In [ ]:
f = pi ; f

pi

In [ ]:
f.parent()

Symbolic Ring

In [ ]:
g = e+f ; g

pi + 1.614213562373095

In [ ]:
g.parent()

Symbolic Ring

But you might want to swim upstream. Given a floating point
approximation of a real number, there are various ways to recover an
exact representation it *may* come from.

### From numerical to rational

An element `x` of `RealField(prec)` can be transformed into a rational
in different ways:

-   `x.exact_rational()` returns the exact rational representation of
    the floating-point number `x`. In particular its denominator is a
    power of
    1.  It contains the same information as `.sign_mantissa_exponent` :

In [ ]:
a = RR(pi) ; a

3.14159265358979

In [ ]:
a.sign_mantissa_exponent()

(1, 7074237752028440, -51)

In [ ]:
s,m,e = _
s*m*2^(e)

884279719003555/281474976710656

In [ ]:
b = a.exact_rational() ; b

884279719003555/281474976710656

-   `x.simplest_rational()` returns the simplest rational (i.e. with
    smallest denominator) that will become equal to `x` when converted
    into the parent of `x` :

In [ ]:
a = RR(pi) ; a

3.14159265358979

In [ ]:
b = a.simplest_rational() ; b

245850922/78256779

In [ ]:
a == b

True

In [ ]:
RR(b)

3.14159265358979

-   `nearby_rational(max_error=e)` finds the simplest rational that is
    at distance at most `e` of `x` :

In [ ]:
RR(pi).nearby_rational(max_error=0.01)

22/7

-   `nearby_rational(max_denominator=d)` finds the closest rational to
    `x` with denominator at most `d` :

In [ ]:
RR(pi).nearby_rational(max_denominator=120)

355/113

### From numerical to algebraic

If `x` is an element of `RealField(prec)`, `x.algebraic_dependency(d)`
returns an integer polynomial `P` of degree at most `d` such that `P(x)`
is almost zero:

In [ ]:
a = 3.0.nth_root(3)
a += 5*a.ulp()
a.algebraic_dependency(3)

x^3 - 3

### From algebraic field to (smaller, faster) embedded number fields

Algebraic numbers have an `as_number_field_element` method:

In [ ]:
a = QQbar(2).sqrt() + QQbar(3).nth_root(3)
a.as_number_field_element()

(Number Field in a with defining polynomial y^6 - 6*y^4 - 6*y^3 + 12*y^2 - 36*y + 1,
 96/755*a^5 - 54/755*a^4 + 128/151*a^3 + 936/755*a^2 - 1003/755*a + 2184/755,
 Ring morphism:
   From: Number Field in a with defining polynomial y^6 - 6*y^4 - 6*y^3 + 12*y^2 - 36*y + 1
   To:   Algebraic Real Field
   Defn: a |--> -0.02803600793431334?)


In [ ]:
field, number, morphism = _
morphism(number) == a

True

For more than one algebraic number, the previous method generalizes with
the `number_field_elements_from_algebraics` function:

In [ ]:
from sage.rings.qqbar import number_field_elements_from_algebraics
sqrt2 = AA(2).sqrt()
sqrt3 = AA(3).sqrt()
number_field_elements_from_algebraics([sqrt2, sqrt3])

(Number Field in a with defining polynomial y^4 - 4*y^2 + 1,
 [-a^3 + 3*a, -a^2 + 2],
 Ring morphism:
   From: Number Field in a with defining polynomial y^4 - 4*y^2 + 1
   To:   Algebraic Real Field
   Defn: a |--> 0.5176380902050415?)

In [ ]:
field, numbers, morphism = _
morphism(numbers[0]) == sqrt2

True

### From algebraic to symbolic

Algebraic numbers have a `radical_expression` method:

In [ ]:
x = polygen(QQ,'x')
P = x^3-x^2-x-1
P

x^3 - x^2 - x - 1

In [ ]:
P.roots(QQbar)[0][0]

1.839286755214161?

In [ ]:
_.radical_expression()

(1/9*sqrt(11)*sqrt(3) + 19/27)^(1/3) + 4/9/(1/9*sqrt(11)*sqrt(3) + 19/27)^(1/3) + 1/3

Note

Note that the `radical_expression` method does not implement the whole
Galois theory yet, meaning that some algebraic numbers that can be
represented by radicals, might not be modified that way:

In [ ]:
a = QQbar(2).sqrt() + QQbar(3).nth_root(3)
a.radical_expression()

2.856463132680504?

### Exercise: reverse engineering

Suppose that, with a numerical experiment, you have approached an
irrational algebraic number by:

In [ ]:
a = 3.14626436994197234

What is this algebraic number likely to be ?

In [ ]:
# edit here

## Complex numbers

Most real number representation have their complex equivalent.

-   `RR` becomes `CC=ComplexField()`
-   `RDF` becomes `CDF=ComplexDoubleField()`
-   `RIF` becomes `CIF=ComplexIntervalField()`
-   `RBF` becomes `CBF=ComplexBallField()`
-   `QQ` becomes `QQ[I]` (Gauss integers)
-   `AA` becomes `QQbar` (the algebraic closure of `QQ`)
-   `SR` remains `SR` (which contains much more than just complex
    numbers)

Note that by default, `I` and `i` belong to the `Symbolic Ring`, so you
will have to convert it, e.g. `QQbar(I)`, `CDF(I)`, ... or obtain it as
`QQbar.gen()`, `QQbar.gen()`, ... . If you overwrite it, you can recover
it with:

In [ ]:
from sage.symbolic.all import i as I

### Diamonds are not round

We saw previously that contracting makes computations safe. Consider the
following example:

In [ ]:
a = CBF(1/10+I*4/5)
b = CBF((1+I)*2/3)
a.diameter()

4.4408921e-16

In [ ]:
b.diameter()

4.4408921e-16

In [ ]:
b.abs() < 1

True

In [ ]:
(a*b).diameter()

1.1768364e-15

Explain this loss of precision.

*Hint*: draw a picture !

Complex ball elements are pairs of real ball elements, see
<http://arblib.org/acb.html>

## Transitional representations (under the hood)

Some "transitional representations" were not included in the big zoo for
simplicity, and because they exist mostly as wrappers for internal
stuff.

### Real literals: between your fingers and Sage

When the user writes:

In [ ]:
a = 0.1

this defines a floating-point number though the precision is not
explicitely set.

Sage defines some default precision (depending on the number of digits
that are provided, 53 in the current case):

In [ ]:
type(a)

<type 'sage.rings.real_mpfr.RealLiteral'>

In [ ]:
a.parent()

Real Field with 53 bits of precision

but it allows casting into higher precision rings, without losing the
precision from the default choice:

In [ ]:
RealField(200)(0.1)

0.10000000000000000000000000000000000000000000000000000000000

In [ ]:
RealField(200)(RDF(0.1))

0.10000000000000000555111512312578270211815834045410156250000

### Real Lazy Field: between exact and numerical representations

Elements of `RLF = RealLazyField()` behaves like floating-point numbers,
This represen

while

`RLF = RealLazyField()` is lazy, in the sense that it doesn't really do
anything but simply sits between exact rings of characteristic 0 and the
real numbers. The values are actually computed when they are cast into a
field of fixed precision.

`RLF = RealLazyField()` like `RealField()` but can wait that the user
fixes the precision to get one.
<http://www.sagemath.org/doc/reference/rings_numerical/sage/rings/real_lazy.html>

In [ ]:
a = RLF(pi) + 2
a

5.141592653589794?

In [ ]:
RealField(100)(a)

5.1415926535897932384626433833

In [ ]:
RealField(150)(a)

5.1415926535897932384626433832795028841971694

The precision of `a` can be increased on demand since, internally, `a`
keeps the exact representation it comes from:

In [ ]:
a._value

pi + 2

Its use is sometimes tricky, and shares some weirdnesses of the symbolic
ring since it also warps its elements:

In [ ]:
RLF(pi) + RLF(QQbar(2).sqrt())

4.555806215962889?

In [ ]:
_._value

pi + 1.414213562373095?